# Proyecto Machine Learning
## Clasificación de pobreza de casas en Costa Rica
### Luis Morera Claramunt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Leyendo el set de datos de entrenamiento

In [50]:
data = pd.read_csv("https://raw.githubusercontent.com/luismoreracla/DataScienceTEC/master/Proyecto/train.csv")

## 1. Preprocesado de datos

In [24]:
data.shape

(9557, 143)

In [25]:
data.describe()

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
count,2.697000e+03,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,2215.000000,9557.000000,9557.000000,...,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9557.000000,9552.000000,9557.000000,9557.000000
mean,1.652316e+05,0.038087,4.955530,0.023648,0.994768,0.957623,0.231767,1.404063,0.385895,1.559171,...,74.222769,1643.774302,19.132887,53.500262,3.844826,3.249485,3.900409,102.588867,1643.774302,3.302292
std,1.504571e+05,0.191417,1.468381,0.151957,0.072145,0.201459,0.421983,0.763131,0.680779,1.036574,...,76.777549,1741.197050,18.751395,78.445804,6.946296,4.129547,12.511831,93.516890,1741.197050,1.009565
min,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,1.000000
25%,8.000000e+04,0.000000,4.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,16.000000,289.000000,9.000000,0.000000,0.000000,1.000000,0.111111,36.000000,289.000000,3.000000
50%,1.300000e+05,0.000000,5.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,36.000000,961.000000,16.000000,36.000000,1.000000,2.250000,0.444444,81.000000,961.000000,4.000000
75%,2.000000e+05,0.000000,6.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,2.000000,...,121.000000,2601.000000,25.000000,81.000000,4.000000,4.000000,1.777778,134.560010,2601.000000,4.000000
max,2.353477e+06,1.000000,11.000000,1.000000,1.000000,1.000000,1.000000,6.000000,5.000000,8.000000,...,441.000000,9409.000000,169.000000,441.000000,81.000000,36.000000,64.000000,1369.000000,9409.000000,4.000000


Las siguientes son columnas que presentan datos faltantes

In [51]:
data.columns[data.isnull().any()].tolist()

['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned']

Se renombran los nombres de la columna para una mejor comprension

In [52]:
data.rename(columns={"v2a1":"Pago renta","v18q1":"# tablets", "rez_esc":"Años retraso escuela", "meaneduc":"Prom de años escuela adultos", "SQBmeaned":"Prom de años escuela adultos cuadrado" }, inplace=True)

In [53]:
data.columns[data.isnull().any()].tolist()

['Pago renta',
 '# tablets',
 'Años retraso escuela',
 'Prom de años escuela adultos',
 'Prom de años escuela adultos cuadrado']

Se calcula cuantos datos faltantes hay en cada columna

In [54]:
data[['Pago renta',
 '# tablets',
 'Años retraso escuela',
 'Prom de años escuela adultos',
 'Prom de años escuela adultos cuadrado']].isnull().sum()

Pago renta                               6860
# tablets                                7342
Años retraso escuela                     7928
Prom de años escuela adultos                5
Prom de años escuela adultos cuadrado       5
dtype: int64

Para los dos features (Promedio de años de escuela en adultos y Promedio de escuela de adultos al cuadrado) que hay 5 datos faltantes, se va a eliminar las instancias donde se encuentren dichos datos faltantes. 

Mientras que para el resto de features, los datos faltantes representan mas de un 70% del numero total de instancias por lo que buscar algun metodo para suponer un valor no se considera confiable, es por esto que se decide no tomar en cuenta dichos features.

Se eliminan los features mencionados anteriormente

In [55]:
data = data.drop(columns = ['Pago renta','# tablets','Años retraso escuela'])

In [56]:
data.shape

(9557, 140)

Se eliminan las instancias que restan con datos faltantes

In [59]:
data.dropna(inplace=True)
data.shape

(9552, 140)

Se comprueba que no hayan datos faltantes

In [61]:
print("Total de datos faltantes: ",data.isna().sum().sum())

Total de datos faltantes:  0


Se comprueba que los datos sean numericos

In [66]:
data.dtypes

Id                                        object
hacdor                                     int64
rooms                                      int64
hacapo                                     int64
v14a                                       int64
refrig                                     int64
v18q                                       int64
r4h1                                       int64
r4h2                                       int64
r4h3                                       int64
r4m1                                       int64
r4m2                                       int64
r4m3                                       int64
r4t1                                       int64
r4t2                                       int64
r4t3                                       int64
tamhog                                     int64
tamviv                                     int64
escolari                                   int64
hhsize                                     int64
paredblolad         

Se calcula la correlacion de variables para ver los features de mas peso dentro del set de datos

In [71]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [76]:
X = data.iloc[:, 0:139]
y = data.iloc[:, -1]